In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
import gc
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('agg')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
!pip install -U git+https://github.com/twopirllc/pandas-ta
import pandas_ta as pta


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 17.1 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:

def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data["6_rsi"] =  pta.rsi(data['Close'], length = 6)
    data["14_rsi"] = pta.rsi(data['Close'], length = 14)
    data["26_rsi"] = pta.rsi(data['Close'], length = 26)
    data["52_rsi"] = pta.rsi(data['Close'], length = 52)

    # drop Adj Close and Volume
    data.drop(columns=['Adj Close', 'Volume'], inplace=True)
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","6_rsi", "14_rsi", "26_rsi","52_rsi"])
        del rows
        del next_candles
        dfs.append(x)
        del x
    del data

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        
        apds = [ 
         mpl.make_addplot(df["6_rsi"],color='b',width=1),
         mpl.make_addplot(df["14_rsi"],color='g',width=1),
         mpl.make_addplot(df["26_rsi"],color='r',width=1),
         mpl.make_addplot(df["52_rsi"],color='y',width=1),
       ]


        mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            savefig=address, tight_layout=True,figsize =(3,2), addplot=apds)
        
        plt.close("all")
        del df
        del date_list
        del rrng
        del i
        del right_now
        gc.collect()



        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 160 == 0:
          print(" ")

    print("")
    del dfs
    del suggs
    del index
    del all_n
    gc.collect()



def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd","FTT-USD"]
download_data(symbols,"max","1wk")

Symbols : 1500
 


In [4]:
how_many_future_candles = 7
how_many_past_candles = 3
each_row_past = 120

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 7
Processing File:
File Number 1: 10/119 20/119 30/119 40/119 50/119 60/119 70/119 80/119 90/119 100/119 110/119 
File Number 2: 10/29 20/29 
File Number 3: 10/119 20/119 30/119 40/119 50/119 60/119 70/119 80/119 90/119 100/119 110/119 
File Number 4: 10/119 20/119 30/119 40/119 50/119 60/119 70/119 80/119 90/119 100/119 110/119 
File Number 5: 10/283 20/283 30/283 40/283 50/283 60/283 70/283 80/283 90/283 100/283 110/283 120/283 130/283 140/283 150/283 160/283  
170/283 180/283 190/283 200/283 210/283 220/283 230/283 240/283 250/283 260/283 270/283 280/283 
File Number 6: 10/283 20/283 30/283 40/283 50/283 60/283 70/283 80/283 90/283 100/283 110/283 120/283 130/283 140/283 150/283 160/283  
170/283 180/283 190/283 200/283 210/283 220/283 230/283 240/283 250/283 260/283 270/283 280/283 
File Number 7: 10/119 20/119 30/119 40/119 50/119 60/119 70/119 80/119 90/119 100/119 110/119 
 


1071

In [5]:
#folder_name = '122227'



label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name)
    dim = (256, 128)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)

data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")

nytrain = []
nytest = []
yn = 0
nn = 0

for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1

yTrain = np.array(nytrain)
yTest = np.array(nytest)

print(f"yn: {yn} nn: {nn}")

(1071, 128, 256, 3)
xTrain : 856 \ xTest : 215
yn: 621 nn: 450


In [6]:
model = Sequential()

model.add(Conv2D(100,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],3)))
model.add(Conv2D(64,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(128,    (3, 3),activation="relu",))
model.add(Conv2D(100,    (2, 2),activation="relu",))
model.add(Conv2D(64,    (1, 1),activation="relu",))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64,    (3, 3),activation="relu",))
model.add(Conv2D(64,    (2, 2),activation="relu",))
model.add(Conv2D(64,    (1, 1),activation="relu",))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 254, 100)     2800      
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 253, 64)      25664     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 41, 84, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 82, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 38, 81, 100)       51300     
                                                                 
 conv2d_4 (Conv2D)           (None, 38, 81, 64)        6464      
                                                        

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=16,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
54/54 [==============================] - 191s 4s/step - loss: 0.6860 - accuracy: 0.5584 - val_loss: 0.6791 - val_accuracy: 0.5814
Epoch 2/20
54/54 [==============================] - 189s 4s/step - loss: 0.6802 - accuracy: 0.5794 - val_loss: 0.6842 - val_accuracy: 0.5814
Epoch 3/20
54/54 [==============================] - 190s 4s/step - loss: 0.6771 - accuracy: 0.5794 - val_loss: 0.6691 - val_accuracy: 0.5814
Epoch 4/20
54/54 [==============================] - 188s 3s/step - loss: 0.6721 - accuracy: 0.5794 - val_loss: 0.6319 - val_accuracy: 0.5814
Epoch 5/20
54/54 [==============================] - 190s 4s/step - loss: 0.5581 - accuracy: 0.7150 - val_loss: 0.5101 - val_accuracy: 0.7442
Epoch 6/20
54/54 [==============================] - 188s 3s/step - loss: 0.4064 - accuracy: 0.8283 - val_loss: 0.4109 - val_accuracy: 0.8465
Epoch 7/20
54/54 [==============================] - ETA: 0s - loss: 0.3010 - accuracy: 0.8692